## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Blind River,CA,46.1834,-82.9582,86.11,48,97,6.53,overcast clouds
1,1,Auki,SB,-8.7676,160.7034,81.54,74,13,2.93,few clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,69,0,9.22,clear sky
3,3,Kalulushi,ZM,-12.8378,28.0919,56.26,65,12,4.32,few clouds
4,4,Bambous Virieux,MU,-20.3428,57.7575,70.21,83,40,12.66,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Blind River,CA,46.1834,-82.9582,86.11,48,97,6.53,overcast clouds
1,1,Auki,SB,-8.7676,160.7034,81.54,74,13,2.93,few clouds
8,8,Shimoda,JP,34.6667,138.9500,80.10,78,100,6.15,overcast clouds
10,10,Jackson,US,42.4165,-122.8345,89.15,29,0,3.44,clear sky
11,11,Guerrero Negro,MX,27.9769,-114.0611,75.81,58,11,16.04,few clouds
13,13,Hasaki,JP,35.7333,140.8333,75.60,83,99,10.22,overcast clouds
19,19,Tabuk,PH,17.4189,121.4443,75.56,91,99,1.92,overcast clouds
24,24,Jamestown,US,42.0970,-79.2353,80.51,73,75,6.91,light rain
32,32,Georgetown,MY,5.4112,100.3354,78.73,94,40,6.91,scattered clouds
35,35,Padang,ID,-0.9492,100.3543,75.25,85,100,4.50,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows. Saw this on Geeks for Geeks video
preferred_cities_df.isnull().sum().sum()


0

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Blind River,CA,86.11,overcast clouds,46.1834,-82.9582,
1,Auki,SB,81.54,few clouds,-8.7676,160.7034,
8,Shimoda,JP,80.10,overcast clouds,34.6667,138.9500,
10,Jackson,US,89.15,clear sky,42.4165,-122.8345,
11,Guerrero Negro,MX,75.81,few clouds,27.9769,-114.0611,
13,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,
19,Tabuk,PH,75.56,overcast clouds,17.4189,121.4443,
24,Jamestown,US,80.51,light rain,42.0970,-79.2353,
32,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,
35,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')    
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Blind River,CA,86.11,overcast clouds,46.1834,-82.9582,Auberge Eldo Inn
1,Auki,SB,81.54,few clouds,-8.7676,160.7034,Auki Motel
8,Shimoda,JP,80.10,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
10,Jackson,US,89.15,clear sky,42.4165,-122.8345,Agate Village Apartments
11,Guerrero Negro,MX,75.81,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
13,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
19,Tabuk,PH,75.56,overcast clouds,17.4189,121.4443,Golden Berries Hotel and Convention
24,Jamestown,US,80.51,light rain,42.0970,-79.2353,Hampton Inn & Suites Jamestown
32,Georgetown,MY,78.73,scattered clouds,5.4112,100.3354,Cititel Penang
35,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,Mercure Padang


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)



In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))